In [1]:
import pandas as pd
import numpy as np
import re
from keras.layers import Dense, Dropout, GRU, Embedding 
from keras.layers import Input, Activation, concatenate, GlobalAveragePooling1D
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras.utils import np_utils, get_custom_objects
from keras.preprocessing import text, sequence
from string import ascii_letters, punctuation, digits

def swish(x):
    return (K.sigmoid(x) * x)

get_custom_objects().update({'swish': Activation(swish)})

SEQ_LENGTH = 100
EMBED_SIZE = 100
VOCAB = 238590
np.random.seed(2017)

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### model

In [2]:
def define_model(matrix):
    rnn = {}
    rnn['units'] = 50
    rnn['return_sequences'] = True
    rnn['recurrent_dropout'] = 0.2
    rnn['dropout'] = 0.1
    rnn['activation'] = 'tanh'
    inputs = Input(shape=(SEQ_LENGTH,), name='sequence')
    embed = Embedding(VOCAB,EMBED_SIZE, weights=[matrix], trainable=False)(inputs)
    lstm = Bidirectional(GRU(**rnn))(embed)
    lstm = BatchNormalization()(lstm)
    max_pool = GlobalMaxPooling1D()(lstm)
    avg_pool = GlobalAveragePooling1D()(lstm)
    pool = concatenate([max_pool, avg_pool])
    pool = BatchNormalization()(pool)
    lstm = Dropout(0.2)(pool)
    dense = Dense(200, activation='swish')(lstm)
    dense = Dropout(0.2)(dense)
    predict = Dense(6, activation='sigmoid')(dense)
    model = Model(inputs=[inputs], output=predict)
    optimizer = Adam(lr=1e-3)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

### generator

In [3]:
embeddings_index = {}

f = open('../data/data/fasttext/vector.vec')

for line in f:
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        print(values[0])
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 93651 word vectors.


In [4]:
def dataflow(train_text, valid_text):
    train_text['comment_text'] = train_text['comment_text'].fillna('nan')
    valid_text['comment_text'] = valid_text['comment_text'].fillna('nan')
    train_text = list(train_text['comment_text'].values)
    valid_text = list(valid_text['comment_text'].values)
    tokenizer = text.Tokenizer(lower=True, char_level=False, num_words=100000)
    tokenizer.fit_on_texts(train_text + valid_text)
    word_index = tokenizer.word_index
    print('total words:', len(word_index))
    intersect = 0
    embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            intersect += 1
    print('common words:', intersect)
    train_token = tokenizer.texts_to_sequences(train_text)
    valid_token = tokenizer.texts_to_sequences(valid_text)
    train_seq = sequence.pad_sequences(train_token, maxlen=SEQ_LENGTH)
    valid_seq = sequence.pad_sequences(valid_token, maxlen=SEQ_LENGTH)
    return train_seq, valid_seq, embedding_matrix

def callbacks(suffix):
    stop = EarlyStopping('val_loss', patience=5, mode="min")
    path = '../data/data/source_1/model_1/model_{}.hdf5'.format(suffix)
    save = ModelCheckpoint(path, save_best_only=True, save_weights_only=True)
    logger = CSVLogger('../data/data/source_1/model_1/logger_{}.log'.format(suffix))
    reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=2, verbose=0, mode='min')
    return [stop, save, reduce, logger]

### data : fold - 1

In [5]:
train_text = pd.read_csv('../data/data/source_1/train/train_data_1.csv')
train_label = pd.read_csv('../data/data/source_1/train/train_labels_1.csv')
valid_text = pd.read_csv('../data/data/source_1/train/test_data_1.csv')
valid_label = pd.read_csv('../data/data/source_1/train/test_labels_1.csv')
train_text, valid_text, embedding_matrix = dataflow(train_text, valid_text)

params = {}
params['x'] = train_text
params['y'] = np.array(train_label.iloc[:,1:])
params['validation_data'] = (valid_text, np.array(valid_label.iloc[:,1:]))
params['batch_size'] = 256
params['epochs'] = 30
params['verbose'] = 1
params['callbacks'] = callbacks(1)
model = define_model(embedding_matrix)
model.fit(**params)

total words: 238589
common words: 88118


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Train on 127656 samples, validate on 31915 samples
Epoch 1/30
127656/127656 [==============================] - 205s 2ms/step - loss: 0.0805 - acc: 0.9724 - val_loss: 0.0500 - val_acc: 0.9815
Epoch 2/30
127656/127656 [==============================] - 202s 2ms/step - loss: 0.0517 - acc: 0.9809 - val_loss: 0.0440 - val_acc: 0.9831
Epoch 3/30
127656/127656 [==============================] - 222s 2ms/step - loss: 0.0487 - acc: 0.9818 - val_loss: 0.0430 - val_acc: 0.9835
Epoch 4/30
127656/127656 [==============================] - 205s 2ms/step - loss: 0.0463 - acc: 0.9826 - val_loss: 0.0424 - val_acc: 0.9833
Epoch 5/30
127656/127656 [==============================] - 205s 2ms/step - loss: 0.0450 - acc: 0.9829 - val_loss: 0.0420 - val_acc: 0.9838
Epoch 6/30
127656/127656 [==============================] - 224s 2ms/step - loss: 0.0442 - acc: 0.9832 - val_loss: 0.0419 - val_acc: 0.9835
Epoch 7/30
127656/127656 [==============================] - 223s 2ms/step - loss: 0.0435 - acc: 0.9833 - val_

### data : fold - 2 

In [7]:
train_text = pd.read_csv('../data/data/source_1/train/train_data_2.csv')
train_label = pd.read_csv('../data/data/source_1/train/train_labels_2.csv')
valid_text = pd.read_csv('../data/data/source_1/train/test_data_2.csv')
valid_label = pd.read_csv('../data/data/source_1/train/test_labels_2.csv')
train_text, valid_text, embedding_matrix = dataflow(train_text, valid_text)

params = {}
params['x'] = train_text
params['y'] = np.array(train_label.iloc[:,1:])
params['validation_data'] = (valid_text, np.array(valid_label.iloc[:,1:]))
params['batch_size'] = 256
params['epochs'] = 30
params['verbose'] = 1
params['callbacks'] = callbacks(2)
model = define_model(embedding_matrix)
model.fit(**params)

total words: 238589
common words: 88118


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Train on 127657 samples, validate on 31914 samples
Epoch 1/30
127657/127657 [==============================] - 212s 2ms/step - loss: 0.0817 - acc: 0.9719 - val_loss: 0.0564 - val_acc: 0.9792
Epoch 2/30
127657/127657 [==============================] - 211s 2ms/step - loss: 0.0515 - acc: 0.9811 - val_loss: 0.0454 - val_acc: 0.9829
Epoch 3/30
127657/127657 [==============================] - 207s 2ms/step - loss: 0.0479 - acc: 0.9820 - val_loss: 0.0463 - val_acc: 0.9816
Epoch 4/30
127657/127657 [==============================] - 199s 2ms/step - loss: 0.0461 - acc: 0.9826 - val_loss: 0.0433 - val_acc: 0.9835
Epoch 5/30
127657/127657 [==============================] - 216s 2ms/step - loss: 0.0449 - acc: 0.9828 - val_loss: 0.0430 - val_acc: 0.9835
Epoch 6/30
127657/127657 [==============================] - 209s 2ms/step - loss: 0.0437 - acc: 0.9833 - val_loss: 0.0433 - val_acc: 0.9836
Epoch 7/30
127657/127657 [==============================] - 214s 2ms/step - loss: 0.0430 - acc: 0.9835 - val_

### data : fold - 3 

In [8]:
train_text = pd.read_csv('../data/data/source_1/train/train_data_3.csv')
train_label = pd.read_csv('../data/data/source_1/train/train_labels_3.csv')
valid_text = pd.read_csv('../data/data/source_1/train/test_data_3.csv')
valid_label = pd.read_csv('../data/data/source_1/train/test_labels_3.csv')
train_text, valid_text, embedding_matrix = dataflow(train_text, valid_text)

params = {}
params['x'] = train_text
params['y'] = np.array(train_label.iloc[:,1:])
params['validation_data'] = (valid_text, np.array(valid_label.iloc[:,1:]))
params['batch_size'] = 256
params['epochs'] = 30
params['verbose'] = 1
params['callbacks'] = callbacks(3)
model = define_model(embedding_matrix)
model.fit(**params)

total words: 238589
common words: 88118


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Train on 127657 samples, validate on 31914 samples
Epoch 1/30
127657/127657 [==============================] - 195s 2ms/step - loss: 0.0787 - acc: 0.9731 - val_loss: 0.0525 - val_acc: 0.9806
Epoch 2/30
127657/127657 [==============================] - 214s 2ms/step - loss: 0.0513 - acc: 0.9809 - val_loss: 0.0455 - val_acc: 0.9831
Epoch 3/30
127657/127657 [==============================] - 215s 2ms/step - loss: 0.0481 - acc: 0.9819 - val_loss: 0.0448 - val_acc: 0.9832
Epoch 4/30
127657/127657 [==============================] - 199s 2ms/step - loss: 0.0464 - acc: 0.9823 - val_loss: 0.0439 - val_acc: 0.9833
Epoch 5/30
127657/127657 [==============================] - 209s 2ms/step - loss: 0.0451 - acc: 0.9827 - val_loss: 0.0439 - val_acc: 0.9832
Epoch 6/30
127657/127657 [==============================] - 214s 2ms/step - loss: 0.0439 - acc: 0.9831 - val_loss: 0.0433 - val_acc: 0.9836
Epoch 7/30
127657/127657 [==============================] - 200s 2ms/step - loss: 0.0431 - acc: 0.9834 - val_

### data : fold - 4

In [9]:
train_text = pd.read_csv('../data/data/source_1/train/train_data_4.csv')
train_label = pd.read_csv('../data/data/source_1/train/train_labels_4.csv')
valid_text = pd.read_csv('../data/data/source_1/train/test_data_4.csv')
valid_label = pd.read_csv('../data/data/source_1/train/test_labels_4.csv')
train_text, valid_text, embedding_matrix = dataflow(train_text, valid_text)

params = {}
params['x'] = train_text
params['y'] = np.array(train_label.iloc[:,1:])
params['validation_data'] = (valid_text, np.array(valid_label.iloc[:,1:]))
params['batch_size'] = 256
params['epochs'] = 30
params['verbose'] = 1
params['callbacks'] = callbacks(4)
model = define_model(embedding_matrix)
model.fit(**params)

total words: 238589
common words: 88118


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Train on 127657 samples, validate on 31914 samples
Epoch 1/30
127657/127657 [==============================] - 196s 2ms/step - loss: 0.0814 - acc: 0.9718 - val_loss: 0.0476 - val_acc: 0.9821
Epoch 2/30
127657/127657 [==============================] - 206s 2ms/step - loss: 0.0516 - acc: 0.9810 - val_loss: 0.0451 - val_acc: 0.9826
Epoch 3/30
127657/127657 [==============================] - 215s 2ms/step - loss: 0.0483 - acc: 0.9821 - val_loss: 0.0438 - val_acc: 0.9830
Epoch 4/30
127657/127657 [==============================] - 208s 2ms/step - loss: 0.0466 - acc: 0.9825 - val_loss: 0.0437 - val_acc: 0.9832
Epoch 5/30
127657/127657 [==============================] - 209s 2ms/step - loss: 0.0450 - acc: 0.9829 - val_loss: 0.0445 - val_acc: 0.9825
Epoch 6/30
127657/127657 [==============================] - 200s 2ms/step - loss: 0.0440 - acc: 0.9832 - val_loss: 0.0429 - val_acc: 0.9834
Epoch 7/30
127657/127657 [==============================] - 214s 2ms/step - loss: 0.0435 - acc: 0.9834 - val_

### data : fold - 5

In [10]:
train_text = pd.read_csv('../data/data/source_1/train/train_data_5.csv')
train_label = pd.read_csv('../data/data/source_1/train/train_labels_5.csv')
valid_text = pd.read_csv('../data/data/source_1/train/test_data_5.csv')
valid_label = pd.read_csv('../data/data/source_1/train/test_labels_5.csv')
train_text, valid_text, embedding_matrix = dataflow(train_text, valid_text)

params = {}
params['x'] = train_text
params['y'] = np.array(train_label.iloc[:,1:])
params['validation_data'] = (valid_text, np.array(valid_label.iloc[:,1:]))
params['batch_size'] = 256
params['epochs'] = 30
params['verbose'] = 1
params['callbacks'] = callbacks(5)
model = define_model(embedding_matrix)
model.fit(**params)

total words: 238589
common words: 88118


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Train on 127657 samples, validate on 31914 samples
Epoch 1/30
127657/127657 [==============================] - 209s 2ms/step - loss: 0.0818 - acc: 0.9718 - val_loss: 0.0516 - val_acc: 0.9809
Epoch 2/30
127657/127657 [==============================] - 198s 2ms/step - loss: 0.0510 - acc: 0.9813 - val_loss: 0.0494 - val_acc: 0.9809
Epoch 3/30
127657/127657 [==============================] - 214s 2ms/step - loss: 0.0479 - acc: 0.9819 - val_loss: 0.0465 - val_acc: 0.9823
Epoch 4/30
127657/127657 [==============================] - 201s 2ms/step - loss: 0.0458 - acc: 0.9826 - val_loss: 0.0457 - val_acc: 0.9829
Epoch 5/30
127657/127657 [==============================] - 214s 2ms/step - loss: 0.0447 - acc: 0.9828 - val_loss: 0.0449 - val_acc: 0.9828
Epoch 6/30
127657/127657 [==============================] - 200s 2ms/step - loss: 0.0434 - acc: 0.9834 - val_loss: 0.0445 - val_acc: 0.9831
Epoch 7/30
127657/127657 [==============================] - 208s 2ms/step - loss: 0.0426 - acc: 0.9835 - val_